In [1]:
import pandas as pd

df = pd.read_csv('/Users/ferdianadham/Downloads/TugasDL/model/joder_reviews_classified_full.csv')

In [2]:
#encode class
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

class_encode = encoder.fit_transform(df['Label Review'])

In [3]:
sentimen_encoder = LabelEncoder()
sentimen_encode = sentimen_encoder.fit_transform(df['Kepuasan Umum/Sentimen'])

In [4]:
df['label encode'] = class_encode
df['Sentimen'] = sentimen_encode

In [5]:
#preprocessing review
df['Review'] = df['Review'].map(lambda x : x.lower() if isinstance(x, str) else x)

In [6]:
import re
def remove_symbol(text):
    return re.sub(r'[^a-zA-Z0-9/]', ' ', text)

In [7]:
df['Review'] = df['Review'].apply(remove_symbol)

In [8]:
df['Review'] = df['Review'].tolist()

In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()

stemmer = factory.create_stemmer()

df['Review'] = [stemmer.stem(doc) for doc in df['Review']]


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=500, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])
padded_sequences = pad_sequences(sequences, maxlen=20)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=500, output_dim=64),
    LSTM(64, return_sequences=False),
    Dense(64, activation="relu"),
    Dense(len(encoder.classes_), activation="softmax"),
])

In [12]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, df['label encode'], test_size=0.2
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    batch_size=16
)

Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3085 - loss: 3.6332 - val_accuracy: 0.3725 - val_loss: 2.3918
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3560 - loss: 2.4194 - val_accuracy: 0.3725 - val_loss: 2.3381
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3923 - loss: 2.2537 - val_accuracy: 0.3725 - val_loss: 2.2572
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3840 - loss: 2.1724 - val_accuracy: 0.3922 - val_loss: 2.1308
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4232 - loss: 1.9698 - val_accuracy: 0.4235 - val_loss: 1.9975
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4885 - loss: 1.7946 - val_accuracy: 0.4588 - val_loss: 1.9448
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5524 - loss: 1.5632 - val_accuracy: 0.4902 - val_loss: 1.9207
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6079 - loss: 1.5206 - val_accuracy: 0.5137 - val_loss:

In [14]:
model_sentimen = Sequential([
    Embedding(input_dim=500, output_dim=64),
    LSTM(64, return_sequences=False),
    Dense(64, activation="relu"),
    Dense(len(sentimen_encoder.classes_), activation="softmax"),
])

In [15]:
model_sentimen.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [16]:
from sklearn.model_selection import train_test_split

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    padded_sequences, df['Sentimen'], test_size=0.2
)

history2 = model_sentimen.fit(
    X_train2, y_train2,
    epochs=50,
    validation_data=(X_test2, y_test2),
    batch_size=16
)

Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5122 - loss: 1.0467 - val_accuracy: 0.5059 - val_loss: 1.0012
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5115 - loss: 0.9646 - val_accuracy: 0.6392 - val_loss: 0.7784
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7907 - loss: 0.5666 - val_accuracy: 0.7569 - val_loss: 0.6639
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8479 - loss: 0.4271 - val_accuracy: 0.7843 - val_loss: 0.6318
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9217 - loss: 0.2568 - val_accuracy: 0.8235 - val_loss: 0.5820
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9239 - loss: 0.1801 - val_accuracy: 0.7529 - val_loss: 0.7628
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9501 - loss: 0.1516 - val_accuracy: 0.8235 - val_loss: 0.7153
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9561 - loss: 0.1098 - val_accuracy: 0.7843 - val_loss:

In [17]:
import numpy as np
contoh_ulasan = ["pesen teh kok gaada tehnya ya"]
seq = tokenizer.texts_to_sequences(contoh_ulasan)
padded = pad_sequences(seq, maxlen=20)

prediksi = model.predict(padded)
kelas = encoder.inverse_transform([np.argmax(prediksi)])
sentimen = model_sentimen.predict(padded)
hasil_sentimen = sentimen_encoder.inverse_transform([np.argmax(sentimen)])
print(kelas)
print(hasil_sentimen)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
['Minuman , Minuman (Es teh)']
['Neutral']


In [19]:
contoh = pd.read_csv('/Users/ferdianadham/Downloads/model_dl/segobabat_reviews.csv')

In [20]:
contoh['Review'] = contoh['Review'].map(lambda x : x.lower() if isinstance(x, str) else x)

In [21]:
contoh['Review'] = contoh['Review'].apply(remove_symbol)

In [22]:
contoh['Review'] = contoh['Review'].tolist()

In [23]:
contoh['Review'] = [stemmer.stem(doc) for doc in contoh['Review']]

In [24]:
tokenizer.fit_on_texts(contoh['Review'])
sequences2 = tokenizer.texts_to_sequences(contoh['Review'])
padded_sequences2 = pad_sequences(sequences2, maxlen=20)

In [25]:
def prediksi_class(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=20)
    pred = model.predict(padded)

    class_index = np.argmax(pred)
    
    if class_index < len(encoder.classes_):
        return encoder.inverse_transform([class_index])[0]
    else:
        return f"Label tidak dikenal: {class_index}"


def prediksi_sentimen(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=20)
    sentimen = model_sentimen.predict(padded)
    
    idx = np.argmax(sentimen)

    if idx < len(sentimen_encoder.classes_):
        return sentimen_encoder.inverse_transform([idx])[0]
    else:
        return f"Label tidak dikenali: {idx}"



In [26]:
contoh['kelas'] = [prediksi_class(doc) for doc in contoh['Review']]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

In [27]:
contoh['sentimen'] = [prediksi_sentimen(doc) for doc in contoh['Review']]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━